In [135]:
from bs4 import BeautifulSoup

import requests
import pandas as pd


df=pd.DataFrame({'link':['abc'],'title':['bcd'], 'date_published':['none'], 'Isselected':['not yet'], 'Tweet':['abc'], 'Summary':['abc']} )
df1 = pd.DataFrame()


In [136]:
#connecting to google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'rkg-gsheet-7ec9124df833.json', scope) # Your json file here

gc = gspread.authorize(credentials)

wks = gc.open('News')

In [137]:
wsheet1 = wks.worksheet("Today's news")

In [138]:
df1 = pd.DataFrame(wsheet1.get_all_records()) #extracting the combined sheet to df1

In [139]:
#scraping from entrakr
html_text = requests.get('https://entrackr.com/').text
soup = BeautifulSoup(html_text, 'lxml')
articles = soup.find_all('h3', class_ = 'elementor-heading-title elementor-size-default')
dates = soup.find_all('span', class_='elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-date')       
    
for result in articles :  
    try:
        #print(result.text)
        df['link']=result.a['href']
        df['title']=result.text
        df['Summary']=df['title']
        df1=df1.append(df)
        
        #print(result.a['href'])
    except:
        continue       

        
df1.drop_duplicates(subset= 'title', keep='first', inplace=True, ignore_index=True)



In [140]:
#scraping from techcrunch
html_text = requests.get('https://techcrunch.com/').text
soup = BeautifulSoup(html_text, 'lxml')
articles = soup.find_all('header', class_ ='post-block__header')
#print(articles)
for result in articles:  
    df['date_published'] = result.time.text.strip()
    df['link']=result.h2.a['href']
    df['title']=result.h2.text.strip()
    df['Summary']=df['title']
    df1=df1.append(df)

df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True) 




In [141]:
df1

,link,title,date_published,Isselected,Tweet,Summary
0,https://entrackr.com/2022/07/exotel-records-12...,Exotel records 12 fold jump in profits with fl...,none,not yet,abc,Exotel records 12 fold jump in profits with fl...
1,https://entrackr.com/2022/07/ed-flags-foreign-...,"ED flags foreign transactions, summons CoinDCX...",none,not yet,abc,"ED flags foreign transactions, summons CoinDCX..."
2,https://entrackr.com/2022/07/urban-company-lau...,Urban Company launches restricted stock unit f...,none,Done,abc,Urban Company launches restricted stock unit f...
3,https://entrackr.com/2022/07/milk-mantra-scale...,"Milk Mantra scale remains flat in FY21, turns ...",none,not yet,abc,"Milk Mantra scale remains flat in FY21, turns ..."
4,https://entrackr.com/2022/07/twitter-sues-indi...,Twitter sues Indian government to challenge ce...,none,not yet,abc,Twitter sues Indian government to challenge ce...
...,...,...,...,...,...,...
285,https://techcrunch.com/2022/07/09/another-wild...,Another wild week for Elon Musk,"Jul 09, 2022",not yet,abc,Another wild week for Elon Musk
286,https://techcrunch.com/2022/07/09/this-week-in...,"This Week in Apps: Google battles KakaoTalk, T...","Jul 09, 2022",not yet,abc,"This Week in Apps: Google battles KakaoTalk, T..."
287,https://techcrunch.com/2022/07/09/data-shows-w...,Data shows who has been hit the hardest in the...,"Jul 09, 2022",not yet,abc,Data shows who has been hit the hardest in the...
288,https://techcrunch.com/2022/07/09/from-spinout...,"From spinouts to fundraising to M&A, founders ...","Jul 09, 2022",not yet,abc,"From spinouts to fundraising to M&A, founders ..."


In [142]:
#scraping from yourstory
html_text = requests.get('https://yourstory.com/category/ys-startup').text
soup = BeautifulSoup(html_text, 'lxml')
articles = soup.find_all('div', class_ = 'sc-cbeScs fNvcGj')
for result in articles :
    df['date_published'] = result.div.span.text.strip()
    df['link']= f"https://yourstory.com/{result.a['href']}"
    df['title']=result.a.span.text.strip()
    df['Summary']=df['title']
    df1=df1.append(df)

df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True)


In [143]:
#scraping from vccircle
html_text = requests.get('https://www.vccircle.com/deal-type/venture-capital').text
soup = BeautifulSoup(html_text, 'lxml')
articles = soup.find_all('div', class_ = 'col-sm-8 article-details')
for result in articles :
    time = result.find('span', class_ = 'publish-time')
    df['date_published'] = time.text.strip()
    title = result.find('div', class_ = 'title')
    df['link']= title.a['href']
    df['title']= title.a.text.strip()
    df['Summary']=df['title']
    df1=df1.append(df)
    
df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True) 



In [144]:
#scraping from economictimes
df['date_published'] = 'none'
html_text = requests.get('https://economictimes.indiatimes.com/tech/technology').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('div', class_ = 'desc')
for result in article:  
    df['link']= f"https://economictimes.indiatimes.com/{result.h4.a['href']}"
    df['title']=result.h4.a.text
    df['Summary']=df['title']
    df1=df1.append(df)
  
   
html_text = requests.get('https://economictimes.indiatimes.com/tech/tech-bytes').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('div', class_ = 'desc')
for result in article: 
    df['link']= f"https://economictimes.indiatimes.com/{result.h4.a['href']}"
    df['title']=result.h4.a.text
    df['Summary']=df['title']
    df1=df1.append(df)

html_text = requests.get('https://economictimes.indiatimes.com/tech/funding').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('div', class_ = 'desc')
for result in article:  
    df['link']= f"https://economictimes.indiatimes.com/{result.h4.a['href']}"
    df['title']=result.h4.a.text
    df['Summary']=df['title']
    df1=df1.append(df)   

html_text = requests.get('https://economictimes.indiatimes.com/tech/startups').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('div', class_ = 'desc')
for result in article:     
    df['link']= f"https://economictimes.indiatimes.com/{result.h4.a['href']}"
    df['title']=result.h4.a.text
    df['Summary']=df['title']
    df1=df1.append(df)
    

df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True) 



In [145]:
#scraping from indianstartupnews
html_text = requests.get('https://indianstartupnews.com/').text
soup = BeautifulSoup(html_text, 'lxml')
articles = soup.find_all('h3', class_ = 'entry-title td-module-title')
dates = soup.find_all('time', class_='entry-date updated td-module-date')       

for result in articles :  
    try:
        #print(result.text)
        df['link']=result.a['href']
        df['title']=result.text
        df['Summary']=df['title']
        df1=df1.append(df)
        
        #print(result.a['href'])
    except:
        continue

        
df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True)   


In [146]:
#scraping from investvine
html_text = requests.get('https://investvine.com/highlights/tech-and-education/startups/').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('h3', class_ = 'entry-title')



for result in article:  
    df['link'] = result.a['href']
    df['title'] = result.a.text.strip()
    df['Summary']=df['title']
    df1 = df1.append(df)

df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True)    



In [147]:
#scraping from inc42
html_text = requests.get('https://inc42.com/').text
soup = BeautifulSoup(html_text, 'lxml')

article = soup.find_all('h2', class_ = 'entry-title')
for result in article:
        try:
            #print(result.a.text)
            df['link']=result.a['href']
            df['title']=result.text.strip()
            df['Summary']=df['title']
            df1=df1.append(df)
        except:
            continue
            
df1.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=True)    
df1

,link,title,date_published,Isselected,Tweet,Summary
0,https://entrackr.com/2022/07/exotel-records-12...,Exotel records 12 fold jump in profits with fl...,none,not yet,abc,Exotel records 12 fold jump in profits with fl...
1,https://entrackr.com/2022/07/ed-flags-foreign-...,"ED flags foreign transactions, summons CoinDCX...",none,not yet,abc,"ED flags foreign transactions, summons CoinDCX..."
2,https://entrackr.com/2022/07/urban-company-lau...,Urban Company launches restricted stock unit f...,none,Done,abc,Urban Company launches restricted stock unit f...
3,https://entrackr.com/2022/07/milk-mantra-scale...,"Milk Mantra scale remains flat in FY21, turns ...",none,not yet,abc,"Milk Mantra scale remains flat in FY21, turns ..."
4,https://entrackr.com/2022/07/twitter-sues-indi...,Twitter sues Indian government to challenge ce...,none,not yet,abc,Twitter sues Indian government to challenge ce...
...,...,...,...,...,...,...
329,https://inc42.com/buzz/suspension-of-vaulds-op...,Suspension Of Vauld’s Operations Puts Spotligh...,none,not yet,abc,Suspension Of Vauld’s Operations Puts Spotligh...
330,https://inc42.com/buzz/india-home-to-73k-start...,"India Home To 73K Startups, Generated Over 7.5...",none,not yet,abc,"India Home To 73K Startups, Generated Over 7.5..."
331,https://inc42.com/buzz/malaysia-indonesia-base...,"Malaysia, Indonesia-Based Hacker Groups Call F...",none,not yet,abc,"Malaysia, Indonesia-Based Hacker Groups Call F..."
332,https://inc42.com/buzz/phonepe-google-pay-cont...,"PhonePe, Google Pay Continue Their Dominance, ...",none,not yet,abc,"PhonePe, Google Pay Continue Their Dominance, ..."


In [148]:
#updating df1 to the combined sheet 
wsheet1.clear()

{'spreadsheetId': '1Uy80E7nplKzZVfUdJWDCLoxvloMMQuPDKNjBaxpW384',
 'clearedRange': "'Today''s news'!A1:Z1001"}

In [149]:
wsheet1.update([df1.columns.values.tolist()]+df1.values.tolist())

{'spreadsheetId': '1Uy80E7nplKzZVfUdJWDCLoxvloMMQuPDKNjBaxpW384',
 'updatedRange': "'Today''s news'!A1:F335",
 'updatedRows': 335,
 'updatedColumns': 6,
 'updatedCells': 2010}